In [1]:
import os
import random
import numpy as np
from datetime import datetime
from pathlib import Path
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.backends import mps

import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image

import mlflow
import mlflow.pytorch
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, precision_score, confusion_matrix
from imblearn.metrics import sensitivity_score, specificity_score

In [2]:
def get_device():
    """Get the best available device for training."""
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print(f"CUDA device detected: {torch.cuda.get_device_name()}")
        return device
    elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
        device = torch.device("mps")
        print("MPS device detected (Apple Silicon)")
        return device
    else:
        print("No GPU available, using CPU")
        return torch.device("cpu")

def set_seed(seed=42):
    """Set random seeds for reproducibility."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

In [4]:
class ResNet50FineTuned(nn.Module):
    """ResNet50 with ImageNet pretrained weights, fine-tuning only the last layer."""
    
    def __init__(self, num_classes=2, freeze_features=True):
        super(ResNet50FineTuned, self).__init__()
        
        # Load pretrained ResNet50
        self.resnet50 = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
        
        # Freeze all layers except the final classifier if freeze_features=True
        if freeze_features:
            for param in self.resnet50.parameters():
                param.requires_grad = False
        
        # Replace the final fully connected layer
        # ResNet50 has 2048 features in the final layer
        num_features = self.resnet50.fc.in_features
        self.resnet50.fc = nn.Linear(num_features, num_classes)
        
        # Ensure the final layer is trainable
        for param in self.resnet50.fc.parameters():
            param.requires_grad = True
            
        print(f"ResNet50 loaded with ImageNet weights. Final layer: {num_features} -> {num_classes}")
        if freeze_features:
            print("All layers frozen except final classifier layer")
        else:
            print("All layers trainable")
    
    def forward(self, x):
        return self.resnet50(x)
    
    def get_trainable_params(self):
        """Return only trainable parameters."""
        return [param for param in self.parameters() if param.requires_grad]
    
    def get_num_trainable_params(self):
        """Return number of trainable parameters."""
        return sum(p.numel() for p in self.parameters() if p.requires_grad)


In [5]:
class ChestXrayDataset(Dataset):
    """Custom dataset for chest x-ray classification."""
    def __init__(self, root_dir, transform=None, max_samples_per_class=None):
        """
        Args:
            root_dir (string): Directory with subdirectories for each class (NORMAL, PNEUMONIA)
            transform (callable, optional): Optional transform to be applied on a sample.
            max_samples_per_class (int, optional): Maximum number of samples per class to load.
                                                  If None, loads all samples.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.class_names = []
        self.max_samples_per_class = max_samples_per_class
        
        # Get class directories
        class_dirs = [d for d in os.listdir(root_dir) 
                     if os.path.isdir(os.path.join(root_dir, d))]
        class_dirs.sort()  # Ensure consistent ordering
        self.class_names = class_dirs
        
        print(f"Found {len(class_dirs)} classes: {class_dirs}")
        if max_samples_per_class:
            print(f"Limiting to {max_samples_per_class} samples per class")
        
        # Create mapping from class name to index
        self.class_to_idx = {class_name: idx for idx, class_name in enumerate(class_dirs)}
        
        # Collect all image paths and labels
        for class_name in class_dirs:
            class_dir = os.path.join(root_dir, class_name)
            class_idx = self.class_to_idx[class_name]
            
            # Get all image files for this class
            class_images = []
            for img_name in os.listdir(class_dir):
                if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    class_images.append(os.path.join(class_dir, img_name))
            
            # Limit samples per class if specified
            if max_samples_per_class and len(class_images) > max_samples_per_class:
                # Randomly sample max_samples_per_class images
                import random
                random.seed(42)  # For reproducibility
                class_images = random.sample(class_images, max_samples_per_class)
            
            # Add to dataset
            for img_path in class_images:
                self.image_paths.append(img_path)
                self.labels.append(class_idx)
        
        print(f"Total samples: {len(self.image_paths)}")
        for class_name, class_idx in self.class_to_idx.items():
            count = sum(1 for label in self.labels if label == class_idx)
            print(f"  {class_name}: {count} samples")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]
        
        # Load image
        image = Image.open(image_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        return image, label


In [6]:
def test_model(model, test_loader, epoch: int = -1, dataset: str = "", criterion=None, device=None):
    """Evaluate model performance on test set."""
    model.eval()
    test_loss = 0
    all_predictions = []
    all_labels = []
    all_probabilities = []
    
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc=f"Testing {dataset}"):
            images = images.to(device=device, dtype=torch.float32)
            labels = labels.to(device=device, dtype=torch.long)
            
            outputs = model(images)
            if criterion:
                loss = criterion(outputs, labels)
                test_loss += loss.item()
            
            probabilities = F.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs.data, 1)
            
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probabilities.extend(probabilities.cpu().numpy())
    
    # Calculate metrics
    all_predictions = np.array(all_predictions)
    all_labels = np.array(all_labels)
    all_probabilities = np.array(all_probabilities)
    
    accuracy = accuracy_score(all_labels, all_predictions)
    f1 = f1_score(all_labels, all_predictions, average='weighted')
    
    # For binary classification, calculate additional metrics
    if all_probabilities.shape[1] == 2:
        try:
            auc = roc_auc_score(all_labels, all_probabilities[:, 1])
        except ValueError:
            auc = None
            
        sensitivity = sensitivity_score(all_labels, all_predictions, average='binary')
        specificity = specificity_score(all_labels, all_predictions, average='binary')
        ppv = precision_score(all_labels, all_predictions, average='binary')
    else:
        auc = None
        sensitivity = sensitivity_score(all_labels, all_predictions, average='macro')
        specificity = specificity_score(all_labels, all_predictions, average='macro')
        ppv = precision_score(all_labels, all_predictions, average='weighted')
    
    confusion_mat = confusion_matrix(all_labels, all_predictions)
    test_loss = test_loss / len(test_loader) if criterion else 0
    
    return auc, sensitivity, specificity, accuracy, f1, ppv, confusion_mat, test_loss

In [10]:
def train_resnet50_model(
        root_path: str,
        freeze_features: bool = True,
        normalize_mean: list = [0.485, 0.456, 0.406],
        normalize_std: list = [0.229, 0.224, 0.225],
        epochs: int = 10,
        batch_size: int = 32,
        lr: float = 1e-3,
        seed: int = 42,
        load_path: str = "",
        save_checkpoint: bool = True,
        save_interval: int = 5,
        save_path: str = "./checkpoints",
        use_mlflow: bool = True,
        experiment_name: str = "ResNet50_ChestXray_FineTuning",
        device=None,
        amp: bool = False,
        num_classes: int = 2,
        max_samples_per_class: int = None
):
    """Main training function for ResNet50 fine-tuning."""
    
    if device is None:
        device = get_device()
    
    set_seed(seed)
    
    # Start MLflow run
    if use_mlflow:
        mlflow.set_experiment(experiment_name)
        mlflow.start_run()
        
        # Log parameters
        mlflow.log_param("model_type", "ResNet50_FineTuned")
        mlflow.log_param("freeze_features", freeze_features)
        mlflow.log_param("epochs", epochs)
        mlflow.log_param("batch_size", batch_size)
        mlflow.log_param("learning_rate", lr)
        mlflow.log_param("seed", seed)
        mlflow.log_param("device", device.type)
        mlflow.log_param("amp", amp)
        mlflow.log_param("num_classes", num_classes)
        mlflow.log_param("normalize_mean", normalize_mean)
        mlflow.log_param("normalize_std", normalize_std)
        mlflow.log_param("max_samples_per_class", max_samples_per_class)
    
    # Create model
    model = ResNet50FineTuned(num_classes=num_classes, freeze_features=freeze_features)
    
    # Log model info
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = model.get_num_trainable_params()
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")
    print(f"Percentage of trainable parameters: {100 * trainable_params / total_params:.2f}%")
    
    if use_mlflow:
        mlflow.log_param("total_params", total_params)
        mlflow.log_param("trainable_params", trainable_params)
        mlflow.log_param("trainable_percentage", 100 * trainable_params / total_params)
    
    # Create datasets with ImageNet normalization
    normalize = transforms.Normalize(normalize_mean, normalize_std)
    
    train_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomAffine(degrees=10, translate=(0.02, 0.02)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ])
    
    test_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        normalize,
    ])
    
    # Use chest x-ray dataset structure
    train_dir = os.path.join(root_path, "train")
    test_dir = os.path.join(root_path, "test")
    
    if not all(os.path.exists(path) for path in [train_dir, test_dir]):
        raise FileNotFoundError(f"Required data directories not found in {root_path}. "
                                f"Ensure train/ and test/ directories exist.")
    
    dataset_train = ChestXrayDataset(root_dir=train_dir, transform=train_transform, max_samples_per_class=max_samples_per_class)
    dataset_test = ChestXrayDataset(root_dir=test_dir, transform=test_transform, max_samples_per_class=max_samples_per_class)
    
    train_loader = DataLoader(dataset_train, batch_size, shuffle=True, num_workers=0)
    test_loader = DataLoader(dataset_test, batch_size, shuffle=False, num_workers=0)
    
    # Log dataset information
    if use_mlflow:
        mlflow.log_param("train_size", len(dataset_train))
        mlflow.log_param("test_size", len(dataset_test))
        mlflow.log_param("data_path", root_path)
    
    # Training setup
    info = f'''
        Model: ResNet50 Fine-tuned (ImageNet pretrained)
        Freeze features: {freeze_features}
        Seed: {seed}, Batch size: {batch_size}, Epochs: {epochs}
        Learning rate: {lr}, Data path: {root_path}
        Training size: {len(dataset_train)}, Test size: {len(dataset_test)}
        Device: {device.type}, Save checkpoints: {save_checkpoint}
        Trainable params: {trainable_params:,} / {total_params:,} ({100 * trainable_params / total_params:.1f}%)
    '''
    print(info)
    
    if load_path and os.path.exists(load_path):
        model.load_state_dict(torch.load(load_path, map_location=device))
        print(f"Loaded checkpoint from {load_path}")
        if use_mlflow:
            mlflow.log_param("loaded_checkpoint", load_path)
    
    model.to(device)
    
    # Only optimize trainable parameters
    optimizer = optim.Adam(model.get_trainable_params(), lr=lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=3)
    
    # Use device-specific GradScaler to avoid warnings
    if device.type == 'cuda' and amp:
        grad_scaler = torch.amp.GradScaler('cuda', enabled=True)
    elif device.type == 'cpu' and amp:
        # For CPU, disable AMP as it's not supported
        print("Warning: AMP is not supported on CPU, disabling AMP")
        grad_scaler = torch.amp.GradScaler('cpu', enabled=False)
        amp = False
    else:
        grad_scaler = torch.amp.GradScaler(device.type, enabled=amp)
    
    criterion = nn.CrossEntropyLoss()
    
    best_accuracy = 0.0
    
    # Training loop
    for epoch in range(epochs):
        if optimizer.param_groups[0].get("lr", 0) == 0:
            print("Learning rate reached 0, stopping training")
            break
        
        model.train()
        epoch_loss = 0
        correct_predictions = 0
        total_predictions = 0
        
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
        for batch_idx, (images, labels) in enumerate(progress_bar):
            images = images.to(device=device, dtype=torch.float32, memory_format=torch.channels_last)
            labels = labels.to(device=device, dtype=torch.long)
            
            # Use device-appropriate autocast
            if amp and device.type == 'cuda':
                with torch.autocast(device_type='cuda', dtype=torch.float16):
                    outputs = model(images)
                    loss = criterion(outputs, labels)
            elif amp and device.type == 'cpu':
                # CPU doesn't support autocast with float16, use bfloat16 if available
                with torch.autocast(device_type='cpu', dtype=torch.bfloat16, enabled=torch.cpu.amp.is_autocast_available()):
                    outputs = model(images)
                    loss = criterion(outputs, labels)
            else:
                # No autocast
                outputs = model(images)
                loss = criterion(outputs, labels)
            
            optimizer.zero_grad(set_to_none=True)
            
            if amp:
                grad_scaler.scale(loss).backward()
                grad_scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.get_trainable_params(), 1.0)
                grad_scaler.step(optimizer)
                grad_scaler.update()
            else:
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.get_trainable_params(), 1.0)
                optimizer.step()
            
            epoch_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_predictions += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()
            
            # Update progress bar
            progress_bar.set_postfix({
                'Loss': f'{loss.item():.4f}',
                'Acc': f'{100 * correct_predictions / total_predictions:.2f}%'
            })
        
        mean_loss = epoch_loss / len(train_loader)
        train_accuracy = correct_predictions / total_predictions
        
        print(f"Epoch {epoch+1}: Loss={mean_loss:.4f}, "
              f"Train Acc={train_accuracy:.4f}, LR={optimizer.param_groups[0]['lr']:.6f}")
        
        # Evaluation
        model.eval()
        auc, sensitivity, specificity, test_accuracy, f1, ppv, confusion_mat, test_loss = test_model(
            model, test_loader, epoch, "test", criterion, device)
        
        print(f"Test Results - Accuracy: {test_accuracy:.4f}, F1: {f1:.4f}, "
              f"Sensitivity: {sensitivity:.4f}, Specificity: {specificity:.4f}")
        
        scheduler.step(test_accuracy)
        
        # Log metrics to MLflow
        if use_mlflow:
            mlflow.log_metric("train_loss", mean_loss, step=epoch)
            mlflow.log_metric("train_accuracy", train_accuracy, step=epoch)
            mlflow.log_metric("test_loss", test_loss, step=epoch)
            mlflow.log_metric("test_accuracy", test_accuracy, step=epoch)
            mlflow.log_metric("test_f1", f1, step=epoch)
            mlflow.log_metric("test_sensitivity", sensitivity, step=epoch)
            mlflow.log_metric("test_specificity", specificity, step=epoch)
            mlflow.log_metric("learning_rate", optimizer.param_groups[0]["lr"], step=epoch)
            if auc is not None:
                mlflow.log_metric("test_auc", auc, step=epoch)
            mlflow.log_metric("test_precision", ppv, step=epoch)
        
        # Save best model
        if test_accuracy > best_accuracy:
            best_accuracy = test_accuracy
            if save_checkpoint:
                Path(save_path).mkdir(parents=True, exist_ok=True)
                best_model_path = os.path.join(save_path, f"best_resnet50_finetuned.pth")
                torch.save(model.state_dict(), best_model_path)
                print(f'Best model saved with accuracy: {best_accuracy:.4f}')
                
                # Log best model to MLflow
                if use_mlflow:
                    mlflow.log_artifact(best_model_path, "models")
                    mlflow.log_metric("best_accuracy", best_accuracy)
        
        # Save periodic checkpoints
        if save_checkpoint and (epoch + 1) % save_interval == 0:
            Path(save_path).mkdir(parents=True, exist_ok=True)
            checkpoint_path = os.path.join(save_path,
                                           f"{datetime.now().strftime('%Y%m%d-%H%M%S')}"
                                           f"_epoch{epoch+1}_resnet50_finetuned.pth")
            torch.save(model.state_dict(), checkpoint_path)
            print(f'Checkpoint saved: {checkpoint_path}')
            
            if use_mlflow:
                mlflow.log_artifact(checkpoint_path, "checkpoints")
    
    # Log final model and end MLflow run
    if use_mlflow:
        mlflow.pytorch.log_model(model, "final_model")
        mlflow.end_run()
    
    return model, best_accuracy

In [14]:
def main():
    """Main function demonstrating ResNet50 fine-tuning."""
    
    # Get device and configure AMP accordingly
    device = get_device()
    print(f"Using device: {device}")
    
    # Enable AMP only for CUDA devices
    use_amp = device.type == 'cuda'
    if use_amp:
        print("Mixed precision training enabled (AMP)")
    else:
        print("Mixed precision training disabled (not supported on this device)")
    
    # Configuration for chest x-ray dataset with ResNet50 fine-tuning
    config = {
        "root_path": "C:/Users/sriha/NEU/shlabs/HP_NVIDIA/CellData/chest_xray",  # Updated path for chest x-ray data
        "freeze_features": True,  # Only fine-tune the last layer
        "epochs": 15,  # More epochs for full dataset training
        "batch_size": 32,  # Larger batch size for full dataset
        "lr": 1e-3,  # Higher learning rate for the new classifier layer
        "seed": 42,
        "save_checkpoint": True,
        "save_interval": 5,  # Save every 5 epochs
        "save_path": "./checkpoints",
        "use_mlflow": True,  # Use MLflow for experiment tracking
        "experiment_name": "ResNet50_ChestXray_FineTuning_Full",
        "amp": use_amp,  # Use AMP only if CUDA is available
        "device": device,  # Pass device explicitly
        "normalize_mean": [0.485, 0.456, 0.406],  # ImageNet normalization
        "normalize_std": [0.229, 0.224, 0.225],   # ImageNet normalization
        "num_classes": 2,  # Binary classification (NORMAL vs PNEUMONIA)
        "max_samples_per_class": None,  # Use full dataset - no sample limit
    }
    
    # Train model
    print("Starting ResNet50 fine-tuning on chest x-ray dataset...")
    print("Using ImageNet pretrained weights, fine-tuning only the last layer")
    print("Using full dataset for complete training")
    model, best_accuracy = train_resnet50_model(**config)
    
    print(f"Training completed! Best accuracy: {best_accuracy:.4f}")
    print(f"Model checkpoints saved in: {config['save_path']}")
    print("Check MLflow UI for detailed experiment tracking and metrics visualization.")
    print("Note: This was trained on the full dataset for complete training.")

In [15]:
    import subprocess
    import time

    # Start MLflow UI in the background
    mlflow.end_run()
    mlflow_process = subprocess.Popen([
        'mlflow', 'ui', 
        '--host', '127.0.0.1', 
        '--port', '5000'
    ], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    print("MLflow UI started in background")
    print("Access the dashboard at: http://127.0.0.1:5000")
    print(f"Process ID: {mlflow_process.pid}")
    main() 

2025/05/31 16:58:36 INFO mlflow.tracking.fluent: Experiment with name 'ResNet50_ChestXray_FineTuning_Full' does not exist. Creating a new experiment.


MLflow UI started in background
Access the dashboard at: http://127.0.0.1:5000
Process ID: 24448
CUDA device detected: NVIDIA GeForce RTX 3060 Laptop GPU
Using device: cuda
Mixed precision training enabled (AMP)
Starting ResNet50 fine-tuning on chest x-ray dataset...
Using ImageNet pretrained weights, fine-tuning only the last layer
Using full dataset for complete training
ResNet50 loaded with ImageNet weights. Final layer: 2048 -> 2
All layers frozen except final classifier layer
Total parameters: 23,512,130
Trainable parameters: 4,098
Percentage of trainable parameters: 0.02%
Found 2 classes: ['NORMAL', 'PNEUMONIA']
Total samples: 5232
  NORMAL: 1349 samples
  PNEUMONIA: 3883 samples
Found 2 classes: ['NORMAL', 'PNEUMONIA']
Total samples: 624
  NORMAL: 234 samples
  PNEUMONIA: 390 samples

        Model: ResNet50 Fine-tuned (ImageNet pretrained)
        Freeze features: True
        Seed: 42, Batch size: 32, Epochs: 15
        Learning rate: 0.001, Data path: C:/Users/sriha/NEU/shlab

Epoch 1/15: 100%|███████████████████████████████████████████| 164/164 [02:56<00:00,  1.08s/it, Loss=0.1661, Acc=89.09%]


Epoch 1: Loss=0.2765, Train Acc=0.8909, LR=0.001000


Testing test: 100%|████████████████████████████████████████████████████████████████████| 20/20 [00:18<00:00,  1.10it/s]


...

Epoch 15: Loss=0.1128, Train Acc=0.9591, LR=0.000100


Testing test: 100%|████████████████████████████████████████████████████████████████████| 20/20 [00:13<00:00,  1.50it/s]


Test Results - Accuracy: 0.8926, F1: 0.8895, Sensitivity: 0.9769, Specificity: 0.7521
Checkpoint saved: ./checkpoints\20250531-173956_epoch15_resnet50_finetuned.pth


2025/05/31 17:39:56 WARNING mlflow.utils.requirements_utils: Found torch version (2.5.1+cu121) contains a local version label (+cu121). MLflow logged a pip requirement for this package as 'torch==2.5.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/05/31 17:40:07 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.20.1+cu121) contains a local version label (+cu121). MLflow logged a pip requirement for this package as 'torchvision==0.20.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/05/31 17:40:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Training completed! Best accuracy: 0.9087
Model checkpoints saved in: ./checkpoints
Check MLflow UI for detailed experiment tracking and metrics visualization.
Note: This was trained on the full dataset for complete training.
